# How to convert Axona data to NWB

In this notebook we show how to convert Axona raw (`.bin` + `.set`) or unit data (`.X`, `.pos`, `.eeg`, `.egf`) to NWB format. Since the Hussaini lab already has tools for extracting unit data from raw data, we implemented conversions based on `.eeg` or `.egf` and `.pos` files for LFP and position data respectively, rather than extracting the information directly from the `.bin` file. 

For the `.pos` file data this will allow the option for generating a `.pos` file from another recording setup that can then be added to an `nwb` file.

For convenience we made a few `NWBConverters` with various input files. Later we will also show how to add data from from e.g. a `.pos` file to an existing NWB file using the appropriate converter.

### Setting up the environment

In [1]:
import json
from pathlib import Path

import matplotlib.pyplot as plt
from pynwb import NWBHDF5IO
from nwbwidgets import nwb2widget

In [2]:
from nwb_conversion_tools import (
    NWBConverter,
    AxonaRecordingExtractorInterface,
    AxonaUnitRecordingExtractorInterface,
    AxonaPositionDataInterface,
    AxonaLFPDataInterface
)

In [3]:
# For convenience, make sure to select a set-filename for which we also have raw, tetrode, eeg, egf and pos data.

filename = '/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/axona_sample.set'
filename_eeg = filename.replace('.set', '.eeg')
filename_egf = filename.replace('.set', '.egf')

base_dir = Path(filename).parent

In [4]:
from hussaini_lab_to_nwb import (
    HussainiBinNWBConverter,
    HussainiBinPosLfpNWBConverter,
    HussainiTetrodeNWBConverter,
    HussainiPosNWBConverter,
    HussainiLfpNWBConverter,
    HussainiUnitNWBConverter
)

1. __HussainiBinPosLfpNWBConverter__: `.bin` + `.pos` + `.eeg` + `.set`

In [5]:
# Set nwbfile name
nwbfile_name = 'nwb_test_bin_pos_lfp.nwb'


# Specify source data
source_data = dict(
    AxonaLFPDataInterface=dict(filename=filename_eeg),
    AxonaRecordingExtractorInterface=dict(filename=filename),
    AxonaPositionDataInterface=dict(filename=filename),
)
print(json.dumps(source_data, indent=2))


# Initialize Converter
converter = HussainiBinPosLfpNWBConverter(source_data=source_data)


# Get metadata
metadata = converter.get_metadata()


# Export to NWB file
output_file = base_dir / nwbfile_name

converter.run_conversion(
    metadata=metadata,
    nwbfile_path=output_file,
    overwrite=True,
    save_to_file=True,
    conversion_options=None
)

{
  "AxonaLFPDataInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/axona_sample.eeg"
  },
  "AxonaRecordingExtractorInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/axona_sample.set"
  },
  "AxonaPositionDataInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/axona_sample.set"
  }
}
Source data is valid!
Metadata is valid!


/home/sbuergers/spikeinterface/hussaini-test-pipeline/venv/lib/python3.8/site-packages/pynwb/file.py:801: UserWarning: Date is missing timezone information. Updating to local timezone.
  warn("Date is missing timezone information. Updating to local timezone.")
/home/sbuergers/spikeinterface/hussaini-test-pipeline/venv/lib/python3.8/site-packages/hdmf/common/table.py:447: UserWarning: An attribute 'name' already exists on DynamicTable 'electrodes' so this column cannot be accessed as an attribute, e.g., table.name; it can only be accessed using other methods, e.g., table['name'].
  warn(msg)
/home/sbuergers/spikeinterface/nwb-conversion-tools/nwb_conversion_tools/utils/spike_interface.py:302: UserWarning: cannot create electrodes for this recording as ids already exist
  warnings.warn("cannot create electrodes for this recording as ids already exist")
/home/sbuergers/spikeinterface/hussaini-test-pipeline/venv/lib/python3.8/site-packages/spikeextractors/extraction_tools.py:802: UserWarni

NWB file saved at /mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/nwb_test_bin_pos_lfp.nwb!


In [6]:
# Check NWB file

fname = output_file
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)
    

%matplotlib inline

io = NWBHDF5IO(output_file, mode='r')
nwb = io.read()

nwb2widget(nwb)

root pynwb.file.NWBFile at 0x140555491977728
Fields:
  acquisition: {
    ElectricalSeries_raw <class 'pynwb.ecephys.ElectricalSeries'>
  }
  devices: {
    Axona <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    Group0 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group1 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group2 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group3 <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  experimenter: ['Abid']
  file_create_date: [datetime.datetime(2021, 8, 7, 13, 11, 9, 488518, tzinfo=tzoffset(None, 7200))]
  identifier: 8c286951-10ac-4baf-bcd5-6ce2fe5817a0
  processing: {
    behavior <class 'pynwb.base.ProcessingModule'>,
    ecephys <class 'pynwb.base.ProcessingModule'>
  }
  session_start_time: 2020-10-04 11:07:07+02:00
  timestamps_reference_time: 2020-10-04 11:07:07+02:00



2. __HussainiBinNWBConverter__: `.bin` + `.set`

In [6]:
# Set nwbfile name
nwbfile_name = 'nwb_test_bin.nwb'


# Specify source data
source_data = dict(
    AxonaRecordingExtractorInterface=dict(filename=filename)
)
print(json.dumps(source_data, indent=2))


# Initialize Converter
converter = HussainiBinNWBConverter(source_data=source_data)


# Get metadata
metadata = converter.get_metadata()


# Export to NWB file
output_file = base_dir / nwbfile_name

converter.run_conversion(
    metadata=metadata,
    nwbfile_path=output_file,
    overwrite=True,
    save_to_file=True,
    conversion_options=None
)


# Check NWB file
fname = output_file
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)
    

%matplotlib inline

io = NWBHDF5IO(output_file, mode='r')
nwb = io.read()

nwb2widget(nwb)

{
  "AxonaRecordingExtractorInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/axona_sample.set"
  }
}
Source data is valid!
Metadata is valid!


/home/sbuergers/spikeinterface/hussaini-test-pipeline/venv/lib/python3.8/site-packages/pynwb/file.py:801: UserWarning: Date is missing timezone information. Updating to local timezone.
  warn("Date is missing timezone information. Updating to local timezone.")
/home/sbuergers/spikeinterface/hussaini-test-pipeline/venv/lib/python3.8/site-packages/hdmf/common/table.py:447: UserWarning: An attribute 'name' already exists on DynamicTable 'electrodes' so this column cannot be accessed as an attribute, e.g., table.name; it can only be accessed using other methods, e.g., table['name'].
  warn(msg)


NWB file saved at /mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/nwb_test_bin2.nwb!
root pynwb.file.NWBFile at 0x140559522303040
Fields:
  acquisition: {
    ElectricalSeries_raw <class 'pynwb.ecephys.ElectricalSeries'>
  }
  devices: {
    Axona <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    Group0 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group1 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group2 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group3 <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  experimenter: ['Abid']
  file_create_date: [datetime.datetime(2021, 8, 7, 13, 16, 23, 266145, tzinfo=tzoffset(None, 7200))]
  identifier: 809854b5-5306-4696-bfba-205b584fd5ae
  session_start_time: 2020-10-04 11:07:07+02:00
  timestamps_reference_time: 2020-10-04 11:07:07+02:00



3. __HussainiTetrodeNWBConverter__: `.X` + `.set`

In [7]:
# Set nwbfile name
nwbfile_name = 'nwb_test_tetrode.nwb'


# Specify source data
source_data = dict(
    AxonaUnitRecordingExtractorInterface=dict(filename=filename)
)
print(json.dumps(source_data, indent=2))


# Initialize Converter
converter = HussainiTetrodeNWBConverter(source_data=source_data)


# Get metadata
metadata = converter.get_metadata()


# Export to NWB file
output_file = base_dir / nwbfile_name

converter.run_conversion(
    metadata=metadata,
    nwbfile_path=output_file,
    overwrite=True,
    save_to_file=True,
    conversion_options=None
)


# Check NWB file

fname = output_file
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)
    

%matplotlib inline

io = NWBHDF5IO(output_file, mode='r')
nwb = io.read()

nwb2widget(nwb)

{
  "AxonaUnitRecordingExtractorInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/axona_sample.set"
  }
}
Source data is valid!
Metadata is valid!
NWB file saved at /mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/nwb_test_tetrode.nwb!
root pynwb.file.NWBFile at 0x140558001034720
Fields:
  acquisition: {
    ElectricalSeries_raw <class 'pynwb.ecephys.ElectricalSeries'>
  }
  devices: {
    Axona <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    Group0 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group1 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group2 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group3 <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  experimenter: ['Abid']
  file_create_date: [datetime.datetime(2021, 8, 7, 13, 16, 36, 922929, tzinfo=tzoffset(None, 7200))]
  identifier: 938f0e64-5a7d-4175-832c-d54712e1f055
  session_s

4. __HussainiPosNWBConverter__: `.pos` + `.set`

In [8]:

# Set nwbfile name
nwbfile_name = 'nwb_test_pos.nwb'


# Specify source data
source_data = dict(
    AxonaPositionDataInterface=dict(filename=filename),
)
print(json.dumps(source_data, indent=2))


# Initialize Converter
converter = HussainiPosNWBConverter(source_data=source_data)


# Get metadata
metadata = converter.get_metadata()


# Export to NWB file
output_file = base_dir / nwbfile_name

converter.run_conversion(
    metadata=metadata,
    nwbfile_path=output_file,
    overwrite=True,
    save_to_file=True,
    conversion_options=None
)


# Check NWB file

fname = output_file
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)
    

%matplotlib inline

io = NWBHDF5IO(output_file, mode='r')
nwb = io.read()

nwb2widget(nwb)

{
  "AxonaPositionDataInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/axona_sample.set"
  }
}
Source data is valid!
Metadata is valid!


/home/sbuergers/spikeinterface/hussaini-test-pipeline/venv/lib/python3.8/site-packages/hdmf/build/objectmapper.py:256: DtypeConversionWarning: Spec 'SpatialSeries/timestamps': Value with data type int64 is being converted to data type float64 as specified.
  warnings.warn(full_warning_msg, DtypeConversionWarning)


NWB file saved at /mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/nwb_test_pos.nwb!
root pynwb.file.NWBFile at 0x140557352545776
Fields:
  file_create_date: [datetime.datetime(2021, 8, 7, 13, 16, 42, 465907, tzinfo=tzoffset(None, 7200))]
  identifier: 5d57a843-ece2-42bc-bb07-7c320c689e7c
  processing: {
    behavior <class 'pynwb.base.ProcessingModule'>
  }
  session_description: no description
  session_start_time: 1970-01-01 00:00:00+01:00
  timestamps_reference_time: 1970-01-01 00:00:00+01:00



5. __HussainiLfpNWBConverter__: `.eeg` + `.set`


In [9]:

# Set nwbfile name
nwbfile_name = 'nwb_test_lfp_eeg.nwb'


# Specify source data
source_data = dict(
    AxonaLFPDataInterface=dict(filename=filename_eeg)
)
print(json.dumps(source_data, indent=2))


# Initialize Converter
converter = HussainiLfpNWBConverter(source_data=source_data)


# Get metadata
metadata = converter.get_metadata()


# Export to NWB file
output_file = base_dir / nwbfile_name

converter.run_conversion(
    metadata=metadata,
    nwbfile_path=output_file,
    overwrite=True,
    save_to_file=True,
    conversion_options=None
)


# Check NWB file

fname = output_file
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)
    

%matplotlib inline

io = NWBHDF5IO(output_file, mode='r')
nwb = io.read()

nwb2widget(nwb)

{
  "AxonaLFPDataInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/axona_sample.eeg"
  }
}
Source data is valid!
Metadata is valid!


/home/sbuergers/spikeinterface/hussaini-test-pipeline/venv/lib/python3.8/site-packages/spikeextractors/extraction_tools.py:802: UserWarning: The recording extractor does not have unscaled traces. Returning scaled traces
  warnings.warn("The recording extractor does not have unscaled traces. Returning scaled traces")


NWB file saved at /mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/nwb_test_lfp_eeg.nwb!
root pynwb.file.NWBFile at 0x140557347712784
Fields:
  devices: {
    Axona <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    Group0 <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  experimenter: ['Abid']
  file_create_date: [datetime.datetime(2021, 8, 7, 13, 16, 45, 865855, tzinfo=tzoffset(None, 7200))]
  identifier: 7e033d59-4973-4dc6-a752-8905637f4eb3
  processing: {
    ecephys <class 'pynwb.base.ProcessingModule'>
  }
  session_start_time: 2020-10-04 11:07:07+02:00
  timestamps_reference_time: 2020-10-04 11:07:07+02:00



6. __HussainiLfpNWBConverter__: `.egf` + `.set`


In [10]:

# Set nwbfile name
nwbfile_name = 'nwb_test_lfp_egf.nwb'


# Specify source data
source_data = dict(
    AxonaLFPDataInterface=dict(filename=filename_egf)
)
print(json.dumps(source_data, indent=2))


# Initialize Converter
converter = HussainiLfpNWBConverter(source_data=source_data)


# Get metadata
metadata = converter.get_metadata()


# Export to NWB file
output_file = base_dir / nwbfile_name

converter.run_conversion(
    metadata=metadata,
    nwbfile_path=output_file,
    overwrite=True,
    save_to_file=True,
    conversion_options=None
)


# Check NWB file

fname = output_file
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)
    

%matplotlib inline

io = NWBHDF5IO(output_file, mode='r')
nwb = io.read()

nwb2widget(nwb)

{
  "AxonaLFPDataInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/axona_sample.egf"
  }
}
Source data is valid!
Metadata is valid!
NWB file saved at /mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/nwb_test_lfp_egf.nwb!
root pynwb.file.NWBFile at 0x140557347343616
Fields:
  devices: {
    Axona <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    Group0 <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  experimenter: ['Abid']
  file_create_date: [datetime.datetime(2021, 8, 7, 13, 16, 49, 479659, tzinfo=tzoffset(None, 7200))]
  identifier: 6080d315-77d6-4d22-a6c1-94c06a2359bf
  processing: {
    ecephys <class 'pynwb.base.ProcessingModule'>
  }
  session_start_time: 2020-10-04 11:07:07+02:00
  timestamps_reference_time: 2020-10-04 11:07:07+02:00



7. __HussainiUnitNWBConverter__: `.X` + `.pos` + `.eeg` + `.set`

In [11]:

# Set nwbfile name
nwbfile_name = 'nwb_test_unit_eeg.nwb'


# Specify source data
source_data = dict(
    AxonaLFPDataInterface=dict(filename=filename_eeg),
    AxonaUnitRecordingExtractorInterface=dict(filename=filename),
    AxonaPositionDataInterface=dict(filename=filename),
)
print(json.dumps(source_data, indent=2))


# Initialize Converter
converter = HussainiUnitNWBConverter(source_data=source_data)


# Get metadata
metadata = converter.get_metadata()


# Export to NWB file
output_file = base_dir / nwbfile_name

converter.run_conversion(
    metadata=metadata,
    nwbfile_path=output_file,
    overwrite=True,
    save_to_file=True,
    conversion_options=None
)


# Check NWB file

fname = output_file
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)
    

%matplotlib inline

io = NWBHDF5IO(output_file, mode='r')
nwb = io.read()

nwb2widget(nwb)

{
  "AxonaLFPDataInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/axona_sample.eeg"
  },
  "AxonaUnitRecordingExtractorInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/axona_sample.set"
  },
  "AxonaPositionDataInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/axona_sample.set"
  }
}
Source data is valid!
Metadata is valid!


/home/sbuergers/spikeinterface/nwb-conversion-tools/nwb_conversion_tools/utils/spike_interface.py:302: UserWarning: cannot create electrodes for this recording as ids already exist
  warnings.warn("cannot create electrodes for this recording as ids already exist")


NWB file saved at /mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/nwb_test_unit_eeg.nwb!
root pynwb.file.NWBFile at 0x140557347188896
Fields:
  acquisition: {
    ElectricalSeries_raw <class 'pynwb.ecephys.ElectricalSeries'>
  }
  devices: {
    Axona <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    Group0 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group1 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group2 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group3 <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  experimenter: ['Abid']
  file_create_date: [datetime.datetime(2021, 8, 7, 13, 16, 52, 87744, tzinfo=tzoffset(None, 7200))]
  identifier: 0748ff8f-b98f-478b-a98c-a46ac33a6dc2
  processing: {
    behavior <class 'pynwb.base.ProcessingModule'>,
    ecephys <class 'pynwb.base.ProcessingModule'>
  }
  session_start_time: 2020-10-04 11:07:07+02:00
  timestamps_reference_time: 2020-10-04 11:07

8. __HussainiUnitNWBConverter__: `.X` + `.pos` + `.egf` + `.set`

In [12]:

# Set nwbfile name
nwbfile_name = 'nwb_test_unit_egf.nwb'


# Specify source data
source_data = dict(
    AxonaLFPDataInterface=dict(filename=filename_egf),
    AxonaUnitRecordingExtractorInterface=dict(filename=filename),
    AxonaPositionDataInterface=dict(filename=filename),
)
print(json.dumps(source_data, indent=2))


# Initialize Converter
converter = HussainiUnitNWBConverter(source_data=source_data)


# Get metadata
metadata = converter.get_metadata()


# Export to NWB file
output_file = base_dir / nwbfile_name

converter.run_conversion(
    metadata=metadata,
    nwbfile_path=output_file,
    overwrite=True,
    save_to_file=True,
    conversion_options=None
)


# Check NWB file

fname = output_file
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)
    

%matplotlib inline

io = NWBHDF5IO(output_file, mode='r')
nwb = io.read()

nwb2widget(nwb)

{
  "AxonaLFPDataInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/axona_sample.egf"
  },
  "AxonaUnitRecordingExtractorInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/axona_sample.set"
  },
  "AxonaPositionDataInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/axona_sample.set"
  }
}
Source data is valid!
Metadata is valid!
NWB file saved at /mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/nwb_test_unit_egf.nwb!
root pynwb.file.NWBFile at 0x140557347485056
Fields:
  acquisition: {
    ElectricalSeries_raw <class 'pynwb.ecephys.ElectricalSeries'>
  }
  devices: {
    Axona <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    Group0 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group1 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group2 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group3 <

## Adding data to existing nwb files

Add `.pos` and `.eeg` data to `.bin` acquisition

__Note:__ We did not appropriately close the connection to the nwbfile earlier, and running the below code without doing so will result in an error. To avoid this, simply restart the notebook, evaluate the imports at the beginning and do not evaluate any of the above cells containing inspections of the nwbfiles with nwbwidgets.

In [1]:

# Set nwbfile name
nwbfile_name = 'nwb_test_bin.nwb'


# Specify source data for pos
source_data = dict(
    AxonaPositionDataInterface=dict(filename=filename),
)
print(json.dumps(source_data, indent=2))


# Initialize Converter
converter = HussainiPosNWBConverter(source_data=source_data)


# Get metadata
metadata = converter.get_metadata()


# Export to NWB file
output_file = base_dir / nwbfile_name

converter.run_conversion(
    metadata=metadata,
    nwbfile_path=output_file,
    overwrite=False,
    save_to_file=True,
    conversion_options=None
)



# Specify source data for eeg
source_data = dict(
    AxonaLFPDataInterface=dict(filename=filename_eeg),
)
print(json.dumps(source_data, indent=2))


# Initialize Converter
converter = HussainiLfpNWBConverter(source_data=source_data)


# Get metadata
metadata = converter.get_metadata()


# Export to NWB file
output_file = base_dir / nwbfile_name

converter.run_conversion(
    metadata=metadata,
    nwbfile_path=output_file,
    overwrite=False,
    save_to_file=True,
    conversion_options=None
)


# Check NWB file

fname = output_file
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)
    

%matplotlib inline

io = NWBHDF5IO(output_file, mode='r')
nwb = io.read()

nwb2widget(nwb)

NameError: name 'filename' is not defined

# Intan to NWB

For this there are already resources from the Hussaini lab we should be able to use.

See here: ...

In [ ]:
from nwb_conversion_tools import (
    NWBConverter, AxonaRecordingExtractorInterface, AxonaPositionDataInterface, IntanRecordingInterface
)

In [ ]:
from nwb_conversion_tools import IntanRecordingInterface

In [ ]:
class HussainiIntanNWBConverter(NWBConverter):
    data_interface_classes = dict(
        IntanRecordingInterface=IntanRecordingInterface,
    )

In [ ]:
intan_file = '/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/Intan_data/intan_rhd_test_1.rhd'

In [ ]:
# Specify source data

source_data = dict(
    IntanRecordingInterface=dict(
        file_path=intan_file
    )
)
print(json.dumps(source_data, indent=2))

In [ ]:
# Initialize HussainiIntanNWBConverter

intan_converter = HussainiIntanNWBConverter(source_data=source_data)

In [ ]:
# Get metadata_schema from converter

metadata_schema = intan_converter.get_metadata_schema()

print(json.dumps(metadata_schema['properties'], indent=2))

In [ ]:
# Validate metadata against metadata_schema

validate(
    instance=intan_converter.get_metadata(),
    schema=intan_converter.get_metadata_schema()
)